In [2]:
import pandas as pd
import json
import numpy
from pandas.io.json import json_normalize
from scipy.spatial.distance import cosine
import csv
from osgeo import ogr, gdal
from shapely.geometry import shape
#import fiona


In [3]:
with open('../../data/GeoJSON/milano-grid.geojson') as f:
    grid = json.load(f)

In [116]:
drv1 = ogr.GetDriverByName('GeoJSON')
drv2 = ogr.GetDriverByName('GeoJSON')

In [117]:
# input_layer = ds_in.GetLayer()
# dest_srs = ogr.osr.SpatialReference()
# dest_srs.ImportFromEPSG(4326)
# dest_layer = output_data_source.CreateLayer(table_name,
#                             dest_srs,
#                             input_layer.GetLayerDefn().GetGeomType(),
#                             ['OVERWRITE=YES', 'GEOMETRY_NAME=geom', 'DIM=2', 'FID=id')

In [118]:
ds_in = drv1.Open("../../data/GeoJSON/milano-grid.geojson")
region = drv2.Open("../../data/GeoJSON/milano_census_ace.geojson")

In [119]:
# union1=ogr.Geometry(3)
# union2=ogr.Geometry(3)
# for feat in lyr_in:
#     geom =feat.GetGeometryRef()
#     union1 = union1.Union(geom)
    
# for feat in lyr_region:
#     geom =feat.GetGeometryRef()
#     union2 = union2.Union(geom)



In [120]:
lyr_region = region.GetLayer()
lyr_in = ds_in.GetLayer()

intersections = {} #{cell_id : {region_id: percentge}}

In [105]:
for feature in lyr_region:
    
    geo_ref = feature.GetGeometryRef()
    region_id = feature.GetField('ACE')

    if region_id not in intersections:
        intersections[region_id] = {}

    for poly in lyr_in:       
        poly_ref = poly.GetGeometryRef()
        cell_id = poly.GetField('cellId')

        if geo_ref.Intersects(poly_ref):
            #getting the intersection area
            inter_area = geo_ref.Intersection(poly_ref).GetArea()
            cell_area = poly_ref.GetArea()
            
            #store the proportion
            intersections[region_id][cell_id] = float(inter_area/cell_area)
            
    lyr_in.ResetReading() #Read the road layer from the beginning again

In [106]:
writer = csv.writer(open('intersect.csv', 'w'))
for key, value in intersections.items():
    writer.writerow([key, value])

In [121]:
# cell-based proportions
intersections = {}
lyr_region.ResetReading()
lyr_in.ResetReading()

for feature in lyr_in:    
    geo_ref = feature.GetGeometryRef()
    cell_id = feature.GetField('cellId')
    cell_area = geo_ref.GetArea()
    
    if cell_id not in intersections:
        intersections[cell_id] = {}    

    for poly in lyr_region:       
        poly_ref = poly.GetGeometryRef()        
        region_id = poly.GetField('ACE')
        
        if geo_ref.Intersects(poly_ref):
            
            #getting the intersection area
            inter_area = geo_ref.Intersection(poly_ref).GetArea()            
            
            #store the proportion
            intersections[cell_id][region_id] = float(inter_area/cell_area)
            
    lyr_region.ResetReading() #Read the road layer from the beginning again

In [122]:
writer = csv.writer(open('cell_intersect.csv', 'w'))
for key, value in intersections.items():
    writer.writerow([key, value])